#검증 세트
테스트 세트를 사용하지 안흥면 모델이 과대적합인지 과소 적합인지 판단하기 어렵다. 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누는 것이다. 이런 데이터를 ***검증 세트***라고 한다.

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
###데이터 준비

먼저 훈련 세트와 테스트 세트를 준비한다.

In [2]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

그 다음 훈련 세트를 나누어 훈련세트(sub)와 검증 세트(val)를 만든다.

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


훈련 세트는 4157개가 되었고 검증 세트는 1040개가 되었다.

이제 훈련 세트와 검증 세트를 통해 모델을 만들고 평가해본다.

In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


위와 같이 검증 세트를 사용해 모델을 평가할 수 있다. 하지만 확실히 이 모델을 훈련 세트에 과대적합되어잇다. 매개변수를 바꾸어 더 좋은 모델을 찾아야 할 필요가 있다.


---



#교차 검증
교차 검증은 검증 세트를 때어내어 평가하는 과정을 여러번 반복한다. 그 다음 이 점수를 평균하여 최종 검증 점수를 얻는다.    

훈련 세트를 세 부분으로 나누어 교차검증을 하는 것을 3-폴드 교차 점증이라 한다.   
보통 5폴드 교차검증이나 10-폴드 교차검증을 사용한다.   
사이킷런에는 cross_validate()라는 교차검증 함수가 있다.

In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.03227615, 0.02142096, 0.03553128, 0.02278161, 0.02160478]), 'score_time': array([0.00654936, 0.00178695, 0.00921535, 0.00191951, 0.01514101]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환한다. 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다.     
각 키마다 5개의 숫자가 담겨있다. cross_validate() 함수는 기본저긍로 5-폴드 교차검증을 수행하며 cv매개변수에서 폴드 수를 바꿀수 있다.

In [8]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


위와 같이 교차검증을 수행하여입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있다. 하지만 이 함수는 훈련 세트를 섞어 폴드를 나누지 않는다. 만약 교차검증을 할 때, 훈련 세트를 섞으려면 분할기를 지정해야 한다.     


In [11]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성한다.

In [12]:
  splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state = 42)
  scores = cross_validate(dt, train_input, train_target, cv=splitter)
  print(np.mean(scores['test_score']))

0.8574181117533719


#하이퍼 파라미터 튜닝
하이퍼파라미터란 모델이 학습할 수 없어 사용자가 지정해야만 하는 파라미터를 튜닝하는 것이다.     
먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련하고, 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿔본다.   
이때 max_depth 값과 min_samples_split 값의 최적의 값을 찾아야 하는데 이 두 매개변수는 동시에 바꿔가면서 최적의 값을 찾아야 한다.        
이를 위해 사이킷런에서 제공하는 ***그리드서치***를 사용한다.

In [20]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
###0.0001부터 0.0005까지 0.0001씩 증가하는 5개의 값

In [21]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs=-1)

결정 트리 클래스를 생성하자 마자 바로 그리드 서치 객체에 전달한다. 그 다음으로 일반 모델을 훈련하듯이 gs객체에 fit 메서드를 호출한다. 이 메서드를 호출하면 그리드 서치 객체는 결정 트리 모델의 min_impurity_decrese값을 바꿔가며 총 5번 실행한다.

In [22]:
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


위 코드에서와 같이 검증 점수가 가장 높은 모델의 매개변수 조합으로 훈련된 세트는 gs객체의 best_estimator_속성에 저장되어 있다.이 모델을 일반 결정 트리와 같이 사용할 수 있다.      
또한 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있다.

In [24]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 mean_test_score 키에 저장되어 있다.

In [25]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


위의 값들을 수동으로 고르는 것 보다 넘파이의 argmax() 함수를 사용하여 가장 큰 값의 인덱스를 추출할 수 있다. 그 다음으로 이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다.

In [28]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


이 과정을 정리하면 다음과 같다.

1.   먼저 탐색할 매개변수를 지정한다.
2.   훈련세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수를 조합한다. 이 점수는 그리드 서치 객체에 저장된다.
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장된다.



###조금 더 복잡한 매개변수 조합
결정 트리에서 min_impurity_decrease 는 노드를 분할하기 위한 불순도 감소 최소량을 지정한다. 여기에 max_depth로 트리의 깊이를 제한하고 min_samples_split으로 노드를 나누기 위한 최소 샘플 수도 고른다.

In [42]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), 'max_depth':range(5,20,1), 'min_samples_split':range(2,100,10)}


넘파이의 arange 함수는 첫번째 매개변수 값에서 시작하여 두번째 매개변수에 도달할 때 까지 세번째 매개변수를 계속 더한 배열을 만든다.   
range 함수는 정수만 사용 가능한 함수로, max_depth를 5에서 20가지 1씩 증가시키며, 15개의 값을 만든다. min_samples_split은 2에서 100까지 10씩 증가시키며 10개를 만든다.     
따라서 이 매개변수로 인해 만들어지는 경우의 수는 9 * 15 * 10=1350개이다.     
기본 5-폴드 교차검증을 수행하면 만들어지는 모델의 수는 1350*5 = 6750개이다.

In [30]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [33]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [34]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


위와 같이 GridSearchCV 클래스를 사용해 교차검증을 수행할 수 있다.

#랜덤 서치
매개변수의 값이 수치을 대 값의 범위나 간격을 미리 정하기 어려울 수 있다. 또 많은 매개변수 조건이 있어 그리드 서치 수행시간이 오래 걸릴 수 있다. 이럴때 ***랜덤 서치***를 사용할 수 있다.

In [35]:
from scipy.stats import uniform, randint

In [36]:
rgen=randint(0,10)
rgen.rvs(10)

array([0, 8, 0, 4, 1, 8, 6, 1, 1, 9])

randint 는 정수값을 뽑고 uniform 은 실수 값을 뽑는다.     
아래의 코드는 1000개를 샘플링하여 각 숫자의 개수를 세는 코드이다.

In [37]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 97,  95, 105, 113, 103,  99,  87,  94,  92, 115]))

In [38]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.42910109, 0.33555243, 0.69289825, 0.50384814, 0.66530233,
       0.50394287, 0.77036892, 0.9356387 , 0.07063373, 0.77111103])

이렇게 난수를 발생시키면서 최적의 매개변수를 찾게 할 수 있다.

In [39]:
params = {'min_impurity_decrese': uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples': randint(1,25)}

min_samples_leaf는 리프노드가 되기 위한 최소 샘플의 수이다.    
min_imputiry_decrese는 0.0001에서 0.001 사이의 실수를 샘플링 한다.      
max_depth는 20에서 50 사이의 정수를 샘플링한다.
샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearcCV의 n_iter 매개변수에 지정한다.

In [43]:
from sklearn.model_selection import RandomizedSearchCV
gs=RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': range(5, 20),
                                        'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                                        'min_samples_split': range(2, 100, 10)},
                   random_state=42)

위의 params에 정의된 매개변수 범위에서 총 100번을 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾는다. 그리드 서치보다 훨씬 교차 검증 수를 줄이고 넓은 영역을 탐색할 수 있다.

In [44]:
print(gs.best_params_)

{'min_samples_split': 12, 'min_impurity_decrease': 0.0005, 'max_depth': 11}


아래는 최고의 교차 검증 점수이다.

In [45]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8681935292811135


최적의 모델은 이미 best_estimator_에 저장되어 있다.

In [46]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8615384615384616


수동으로 배개변수를 바꾸는 것 보다 그리드 서치나 랜덤 서치를 사용하여 더 쉽게 테스트 할 수 있다.